In [2]:
from tqdm import tqdm, trange
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import docx
import warnings
import re
import time
from scipy import stats
import wmi
from sys import platform
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.enum.text import WD_BREAK
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
warnings.filterwarnings('ignore')

In [1]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor

##### Характеристика системы

In [3]:
computer = wmi.WMI()
computer_info = computer.Win32_ComputerSystem()[0]
os_info = computer.Win32_OperatingSystem()[0]
proc_info = computer.Win32_Processor()[0]
gpu_info = computer.Win32_VideoController()[0]
os_name = os_info.Name.split('|')[0]
os_version = ' '.join([os_info.Version, os_info.BuildNumber])
system_ram = float(os_info.TotalVisibleMemorySize) / 1048576  # KB to GB

## Предпросмотр данных

In [4]:
interesting_features = ['Мгновенная механическая скорость бурения ', 'Частота вращения шпинделя', 'Индекс углубки', 
                       'Глубина скважины (м)', 'Крутящий момент на шпинделе', 'Давление промывочной жидкости', 
                       'Расход топлива мгновенный в час по шине CAN', 'Расход промывочной жидкости', 'Осевая нагрузка']

In [5]:
check = pd.read_csv('Result_23.csv')

In [6]:
check.shape[0]

3253056

In [7]:
dt = pd.DataFrame()
for feature in tqdm(interesting_features, desc = 'Обработка признаков'):
    dt = dt.append(check[check['DESCRIPTION'] == feature][['VALUE', 'TIMESTAMP', 'DESCRIPTION']])
dt = dt.drop_duplicates()
# dt.to_csv('first_preprocessing.csv', encoding = 'utf-32')
df = dt.pivot_table(index = 'TIMESTAMP', columns = ['DESCRIPTION'], values = ['VALUE'])
df.reset_index(drop = False, inplace = True)
df['TIMESTAMP'] = df['TIMESTAMP'].replace(to_replace = '\.\d\d\d', value = '', regex = True)
timestamp = pd.to_datetime(df['TIMESTAMP'], format = '%Y-%m-%d %H:%M:%S')
df['TIMESTAMP'] = timestamp

Обработка признаков: 100%|███████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  3.73it/s]


In [8]:
print(df.isna().sum().sum())

7015


In [ ]:
total_sec = 60
new = pd.DataFrame()
for i in trange(df.shape[0], desc = 'Обработка пропусков'):
    if i == 0:
        if (df['TIMESTAMP'].iloc[i + 1] - df['TIMESTAMP'].iloc[i]).total_seconds() < total_sec:
            df.iloc[i: i + 2].fillna(method = 'bfill', inplace = True)
    elif i == df.shape[0] - 1:
        if (df['TIMESTAMP'].iloc[i] - df['TIMESTAMP'].iloc[i - 1]).total_seconds() < total_sec:
            df.iloc[i-1:i+1].fillna(method = 'ffill', inplace = True)
    else:
        if (df['TIMESTAMP'].iloc[i] - df['TIMESTAMP'].iloc[i - 1]).total_seconds() < total_sec:
            df.iloc[i-1:i+1].fillna(method = 'ffill', inplace = True)
        if (df['TIMESTAMP'].iloc[i + 1] - df['TIMESTAMP'].iloc[i]).total_seconds() < total_sec:
            df.iloc[i: i + 2].fillna(method = 'bfill', inplace = True)

Обработка пропусков:  74%|████████████████████████████████████████▋              | 67798/91567 [31:17<09:36, 41.26it/s]

In [9]:
print(7015 - df.isna().sum().sum())

3545


In [10]:
df.dropna(inplace = True)

In [16]:
df = pd.read_csv('Чистый.csv', encoding = 'utf-32', index_col = 0, header = 1)
df.rename(columns = {'Unnamed: 1': 'TIMESTAMP'}, inplace = True)

### Детекция на выбросы

In [11]:
interesting_features = list(df.columns)[1:14]

#### z-score

In [12]:
threshold = 2
dict_out_feat_z = {}
for feature in interesting_features:
    z = np.abs(stats.zscore(df[feature]))
    dict_out_feat_z[feature] = set(np.where(z > 2)[0])
set_all = set()
for feature in interesting_features:
    set_all = set_all|dict_out_feat_z[feature]

#### IQR-test

In [13]:
dict_out_feat_iqr = {}
for feature in interesting_features:
    Q1 = np.percentile(df[feature], 25, interpolation='midpoint')
    Q3 = np.percentile(df[feature], 75, interpolation='midpoint')
    IQR = Q3 - Q1
    upper=Q3+1.5*IQR
    lower=Q1-1.5*IQR
    iqr = np.where((df[feature] < lower) | (df[feature] > upper))
    dict_out_feat_iqr[feature] = iqr

### Обучение алгоритма

#### Классический ML

##### Линейные модели

In [20]:
target = [('VALUE','Частота вращения шпинделя')]
rmse = []
mse = []
r2 = []
mae = []
best_estimator = []
estimator = []
times = []
linear_models = [LinearRegression(), Lasso(), Ridge()]

In [21]:
X = df.drop(target + [df.columns[0]], axis = 1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=123)
parameters = {'alpha': np.arange(1, 100, 10)}

In [22]:
model = GridSearchCV(Lasso(), parameters, scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'], refit =  'neg_mean_squared_error', cv = 2, n_jobs = -1)
model.fit(X, y)

GridSearchCV(cv=2, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91])},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error',
                      'r2'])

In [23]:
for linear_model in linear_models:
    linear = linear_model
    start_time = time.time()
    linear.fit(X_train, y_train)
    times.append(time.time() - start_time)
    y_pred = linear.predict(X_test)
    mse.append(round(mean_squared_error(y_test, y_pred), 1))
    rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
    mae.append(round(mean_absolute_error(y_test, y_pred), 1))
    r2.append(round(r2_score(y_test, y_pred), 2))
    best_estimator.append('-')
    estimator.append(f'{str(linear_model).replace("()", "")}')
    if f'{linear_model}' == 'Ridge()' or f'{linear_model}' == 'Lasso()':
        start_time = time.time()
        linear= GridSearchCV(linear_model, parameters, scoring='neg_mean_squared_error',cv=5, n_jobs = -1)
        linear.fit(X_train, y_train)
        times.append(time.time() - start_time)
        y_pred = linear.predict(X_test)
        mse.append(round(mean_squared_error(y_test, y_pred), 1))
        rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
        mae.append(round(mean_absolute_error(y_test, y_pred), 1))
        r2.append(round(r2_score(y_test, y_pred), 2))
        best_estimator.append(f'{linear.best_estimator_}')
        estimator.append(f'{str(linear_model).replace("()", "")}' + " opt")


In [24]:
dict_metrics = {'estimator': estimator,
              'mse': mse,
              'rmse': rmse,
              'r2': r2,
              'mae': mae,
              'best_estimator': best_estimator,
               'time': np.round(times, 1)}
metrics_linear = pd.DataFrame(dict_metrics)

###### Деревянные модели

In [25]:
rmse = []
mse = []
r2 = []
mae = []
best_estimator = []
estimator = []
times = []

parameters_dt = {'max_depth': np.arange(3, 10, 2)}
parameters_rf = {'max_depth': np.arange(3, 10, 4), 'n_estimators': [300]}

# parameters_dt = {'criterion':['squared_error', 'friedman_mse'],'max_depth': np.arange(3, 10),
#               'max_leaf_nodes': np.arange(2, 10, 2), 'min_samples_split': np.arange(2, 5, 1)}
# parameters_rf = {'criterion':['squared_error', 'friedman_mse'],'max_depth': np.arange(3, 10, 2),
#               'max_leaf_nodes': np.arange(2, 10, 4), 'n_estimators': [200, 300]}
trees_models = [DecisionTreeRegressor(), RandomForestRegressor()]

In [26]:
for tree_model in tqdm(trees_models, desc = 'Обучаю модели'):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=123)
    tree = tree_model
    start_time = time.time()
    tree.fit(X_train, y_train)
    times.append(time.time() - start_time)
    y_pred = tree.predict(X_test)
    mse.append(round(mean_squared_error(y_test, y_pred), 1))
    rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
    mae.append(round(mean_absolute_error(y_test, y_pred), 1))
    r2.append(round(r2_score(y_test, y_pred), 2))
    best_estimator.append('-')
    estimator.append(f'{str(tree_model).replace("()", "")}')
    if f'{tree_model}' == 'DecisionTreeRegressor()':
        start_time = time.time()
        tree= GridSearchCV(tree_model, parameters_dt, scoring='neg_mean_squared_error',cv=5, n_jobs = -1)
        tree.fit(X_train, y_train)
        times.append(time.time() - start_time)
        y_pred = tree.predict(X_test)
        mse.append(round(mean_squared_error(y_test, y_pred), 1))
        rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
        mae.append(round(mean_absolute_error(y_test, y_pred), 1))
        r2.append(round(r2_score(y_test, y_pred), 2))
        best_estimator.append(f'{tree.best_estimator_}')
        estimator.append(f'{str(tree_model).replace("()", "")}' + " opt")
    if f'{tree_model}' == 'RandomForestRegressor()':
        start_time = time.time()
        tree= GridSearchCV(tree_model, parameters_rf, scoring='neg_mean_squared_error',cv=5, n_jobs = -1)
        tree.fit(X_train, y_train)
        times.append(time.time() - start_time)
        y_pred = tree.predict(X_test)
        mse.append(round(mean_squared_error(y_test, y_pred), 1))
        rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
        mae.append(round(mean_absolute_error(y_test, y_pred), 1))
        r2.append(round(r2_score(y_test, y_pred), 2))
        best_estimator.append(f'{tree.best_estimator_}')
        estimator.append(f'{str(tree_model).replace("()", "")}' + " opt")

Обучаю модели: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:58<00:00, 89.36s/it]


In [27]:
dict_metrics = {'estimator': estimator,
              'mse': mse,
              'rmse': rmse,
              'r2': r2,
              'mae': mae,
              'best_estimator': best_estimator,
               'time': np.round(times, 1)}
metrics_tree = pd.DataFrame(dict_metrics)

##### Градиентные бустинги

In [28]:
rmse = []
mse = []
r2 = []
mae = []
best_estimator = []
estimator = []
times = []

parameters_gr = {'n_estimators': [500, 1000],
#           'max_depth': [None, 3, 5,],
#           'min_samples_split': [2, 0.3, 0.5, n_samples//2, ],
#           'min_samples_leaf': [1, 0.3, 0.5, n_samples//2, ],
#           'criterion': ['friedman_mse', 'mae'],
#           'max_features': [None, 'sqrt', 'auto', 'log2', 0.3, 0.7, n_features//2, ],
         }
parameters_cat = {'learning_rate': [0.4, 0.1],
        # 'depth': [4, 6, 10],
        #'l2_leaf_reg': [1, 3, 5, 7, 9],
        'iterations': [500, 1000]}

boost_models = [GradientBoostingRegressor(), CatBoostRegressor(iterations=500, max_depth=10, learning_rate = 0.4)]

In [29]:
for boost_model in tqdm(boost_models, desc = 'Обучаю модели'):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=123)
    boost = boost_model
    start_time = time.time()
    boost.fit(X_train, y_train)
    times.append(time.time() - start_time)
    y_pred = boost.predict(X_test)
    mse.append(round(mean_squared_error(y_test, y_pred), 1))
    rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
    mae.append(round(mean_absolute_error(y_test, y_pred), 1))
    r2.append(round(r2_score(y_test, y_pred), 2))
    best_estimator.append('-')
    estimator.append(f'{str(boost_model).replace("()", "")}')
    if f'{boost_model}' == 'GradientBoostingRegressor()':
        start_time = time.time()
        boost= GridSearchCV(boost_model, parameters_gr, scoring='neg_mean_squared_error',cv=5, n_jobs = -1)
        boost.fit(X, y)
        times.append(time.time() - start_time)
        y_pred = boost.predict(X_test)
        mse.append(round(mean_squared_error(y_test, y_pred), 1))
        rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
        mae.append(round(mean_absolute_error(y_test, y_pred), 1))
        r2.append(round(r2_score(y_test, y_pred), 2))
        best_estimator.append(f'{boost.best_estimator_}')
        estimator.append(f'{str(boost_model).replace("()", "")}' + " opt")
    if f'{boost_model}' == 'CatBoostRegressor(iterations=500, max_depth=10, learning_rate = 0.4)':
        start_time = time.time()
        boost = GridSearchCV(boost_model, parameters_cat, scoring='neg_mean_squared_error',cv=5, n_jobs = -1)
        boost.fit(X_train, y_train)
        times.append(time.time() - start_time)
        y_pred = boost.predict(X_test)
        mse.append(round(mean_squared_error(y_test, y_pred), 1))
        rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
        mae.append(round(mean_absolute_error(y_test, y_pred), 1))
        r2.append(round(r2_score(y_test, y_pred), 2))
        best_estimator.append(f'{boost.best_estimator_}')
        estimator.append(f'{str(boost_model).replace("()", "")}' + " opt")

Обучаю модели:  50%|███████████████████████████████████████████████████████████████████████████████▌                                                                               | 1/2 [04:28<04:28, 268.48s/it]

0:	learn: 1869.9016243	total: 186ms	remaining: 1m 32s
1:	learn: 1599.7620665	total: 215ms	remaining: 53.5s
2:	learn: 1403.3476810	total: 243ms	remaining: 40.2s
3:	learn: 1226.9247355	total: 268ms	remaining: 33.2s
4:	learn: 1077.6171043	total: 293ms	remaining: 29s
5:	learn: 976.2775605	total: 317ms	remaining: 26.1s
6:	learn: 887.7006388	total: 342ms	remaining: 24.1s
7:	learn: 796.7963697	total: 368ms	remaining: 22.6s
8:	learn: 719.3466016	total: 395ms	remaining: 21.5s
9:	learn: 657.1726109	total: 427ms	remaining: 20.9s
10:	learn: 599.0021706	total: 452ms	remaining: 20.1s
11:	learn: 544.4676376	total: 475ms	remaining: 19.3s
12:	learn: 496.5318351	total: 500ms	remaining: 18.7s
13:	learn: 454.5142776	total: 524ms	remaining: 18.2s
14:	learn: 411.6091096	total: 548ms	remaining: 17.7s
15:	learn: 380.2857645	total: 573ms	remaining: 17.3s
16:	learn: 344.2855167	total: 598ms	remaining: 17s
17:	learn: 323.4117362	total: 628ms	remaining: 16.8s
18:	learn: 310.8669226	total: 657ms	remaining: 16.6s
1

157:	learn: 74.0820003	total: 4.25s	remaining: 9.19s
158:	learn: 73.9148026	total: 4.27s	remaining: 9.16s
159:	learn: 73.7304628	total: 4.3s	remaining: 9.13s
160:	learn: 73.6269773	total: 4.32s	remaining: 9.1s
161:	learn: 73.3801209	total: 4.35s	remaining: 9.07s
162:	learn: 73.2184320	total: 4.37s	remaining: 9.04s
163:	learn: 73.0385669	total: 4.39s	remaining: 9s
164:	learn: 72.8766025	total: 4.42s	remaining: 8.97s
165:	learn: 72.8003325	total: 4.44s	remaining: 8.94s
166:	learn: 72.6305646	total: 4.47s	remaining: 8.91s
167:	learn: 72.5704036	total: 4.49s	remaining: 8.88s
168:	learn: 72.3532422	total: 4.52s	remaining: 8.85s
169:	learn: 72.1790863	total: 4.54s	remaining: 8.82s
170:	learn: 71.9915390	total: 4.57s	remaining: 8.79s
171:	learn: 71.8667228	total: 4.59s	remaining: 8.76s
172:	learn: 71.6797580	total: 4.62s	remaining: 8.72s
173:	learn: 71.5619179	total: 4.64s	remaining: 8.69s
174:	learn: 71.3925354	total: 4.66s	remaining: 8.66s
175:	learn: 71.2198190	total: 4.69s	remaining: 8.64

315:	learn: 56.0124337	total: 8.21s	remaining: 4.78s
316:	learn: 55.9108885	total: 8.23s	remaining: 4.75s
317:	learn: 55.8618839	total: 8.26s	remaining: 4.72s
318:	learn: 55.8174290	total: 8.28s	remaining: 4.7s
319:	learn: 55.7496201	total: 8.3s	remaining: 4.67s
320:	learn: 55.6798431	total: 8.33s	remaining: 4.64s
321:	learn: 55.6270499	total: 8.35s	remaining: 4.62s
322:	learn: 55.5269554	total: 8.38s	remaining: 4.59s
323:	learn: 55.4761017	total: 8.4s	remaining: 4.57s
324:	learn: 55.4336265	total: 8.43s	remaining: 4.54s
325:	learn: 55.3352124	total: 8.45s	remaining: 4.51s
326:	learn: 55.2868422	total: 8.48s	remaining: 4.49s
327:	learn: 55.2263234	total: 8.5s	remaining: 4.46s
328:	learn: 55.1899151	total: 8.53s	remaining: 4.43s
329:	learn: 55.1496350	total: 8.55s	remaining: 4.41s
330:	learn: 55.0874303	total: 8.58s	remaining: 4.38s
331:	learn: 55.0379009	total: 8.6s	remaining: 4.35s
332:	learn: 54.9764347	total: 8.63s	remaining: 4.33s
333:	learn: 54.9143442	total: 8.65s	remaining: 4.3s

477:	learn: 46.8056036	total: 12.4s	remaining: 568ms
478:	learn: 46.7514060	total: 12.4s	remaining: 543ms
479:	learn: 46.6864711	total: 12.4s	remaining: 517ms
480:	learn: 46.6661757	total: 12.4s	remaining: 491ms
481:	learn: 46.6309588	total: 12.5s	remaining: 465ms
482:	learn: 46.6068510	total: 12.5s	remaining: 439ms
483:	learn: 46.5596323	total: 12.5s	remaining: 413ms
484:	learn: 46.5236076	total: 12.5s	remaining: 388ms
485:	learn: 46.4990609	total: 12.6s	remaining: 362ms
486:	learn: 46.4556532	total: 12.6s	remaining: 336ms
487:	learn: 46.4066410	total: 12.6s	remaining: 310ms
488:	learn: 46.3519422	total: 12.6s	remaining: 284ms
489:	learn: 46.2913574	total: 12.7s	remaining: 258ms
490:	learn: 46.2526780	total: 12.7s	remaining: 233ms
491:	learn: 46.2261978	total: 12.7s	remaining: 207ms
492:	learn: 46.1814517	total: 12.7s	remaining: 181ms
493:	learn: 46.1553139	total: 12.8s	remaining: 155ms
494:	learn: 46.0918240	total: 12.8s	remaining: 129ms
495:	learn: 46.0509426	total: 12.8s	remaining:

Обучаю модели: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [04:41<00:00, 140.93s/it]


In [30]:
dict_metrics = {'estimator': estimator,
              'mse': mse,
              'rmse': rmse,
              'r2': r2,
              'mae': mae,
              'best_estimator': best_estimator,
               'time': np.round(times, 1)}
metrics_boost = pd.DataFrame(dict_metrics)

## Создание документа

In [66]:
doc = docx.Document()
# задаем стиль текста по умолчанию
style = doc.styles['Normal']
# название шрифта
style.font.name = 'Times New Roman'
# размер шрифта
style.font.size = docx.shared.Pt(14)

In [67]:
line_spacing = 1.75
paragraph = doc.add_paragraph('Содержание\n')
run = paragraph.add_run()
fldChar = OxmlElement('w:fldChar')  # creates a new element
fldChar.set(qn('w:fldCharType'), 'begin')  # sets attribute on element
instrText = OxmlElement('w:instrText')
instrText.set(qn('xml:space'), 'preserve')  # sets attribute on element
instrText.text = 'TOC \\o "1-3" \\h \\z \\u'   # change 1-3 depending on heading levels you need

fldChar2 = OxmlElement('w:fldChar')
fldChar2.set(qn('w:fldCharType'), 'separate')
fldChar3 = OxmlElement('w:t')
fldChar3.text = "Right-click to update field."
fldChar2.append(fldChar3)

fldChar4 = OxmlElement('w:fldChar')
fldChar4.set(qn('w:fldCharType'), 'end')
r_element = run._r
r_element.append(fldChar)
r_element.append(instrText)
r_element.append(fldChar2)
r_element.append(fldChar4)
p_element = paragraph._p
doc.add_page_break()
# заголовок 1
head_1 = doc.add_heading('Запрос в SQL', level = 1)
head_1.alignment = WD_ALIGN_PARAGRAPH.CENTER
h_fmt = head_1.paragraph_format
h_fmt.line_spacing = 1.75
# параграф 
paragraph = doc.add_paragraph('SELECT df.ID, TIMESTAMP, dj.DESCRIPTION, VALUE  FROM DRILL_GEO_GAUGE_TRANS df \n\
LEFT JOIN DRILL_GEO_GAUGE_TYPE dj ON dj.ID = df.GAUGE_TYPE_ID \n\
ORDER BY df.ID;')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75

# заголовок_i
head_i = doc.add_heading('Характеристика компьютера', level = 1)
h_fmt = head_i.paragraph_format
h_fmt.line_spacing = 1.75
paragraph = doc.add_paragraph('OS Name: {0}'.format(os_name))
paragraph = doc.add_paragraph('OS Version: {0}'.format(os_version))
paragraph = doc.add_paragraph('CPU: {0}'.format(proc_info.Name))
paragraph = doc.add_paragraph('RAM: {0} GB'.format(system_ram))
paragraph = doc.add_paragraph('Graphics Card: {0}'.format(gpu_info.Name))
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75

# заголовок 2
head_2 = doc.add_heading('Первичный осмотр данных', level = 1)
head_2.alignment = WD_ALIGN_PARAGRAPH.CENTER
h_fmt = head_2.paragraph_format
h_fmt.line_spacing = 1.75
# параграф
paragraph = doc.add_paragraph(f'Всего имеется {check.shape[0]} наблюдений.\n\
Имееются следующие переменные: ')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.5
for var in check['DESCRIPTION'].unique():
    paragraph = doc.add_paragraph(f'{var}', style= 'List Number')
    p_fmt = paragraph.paragraph_format
    p_fmt.line_spacing = 1.75
# табличка
miss = check.isna().sum().reset_index()
table = doc.add_table(rows=check.shape[1] + 1, cols=2, style='Table Grid')
cell = table.cell(0, 0)
cell.paragraphs[0].add_run('Название переменной')
cell = table.cell(0, 1)
cell.paragraphs[0].add_run('Количество пропусков в изначальных данных')
for column, value, index in zip(miss['index'], miss[0], range(1, miss.shape[0] + 1)):
    cell = table.cell(index, 0)
    cell.paragraphs[0].add_run(column)
    cell = table.cell(index, 1)
    cell.paragraphs[0].add_run(f'{value}')
# параграф про дубликаты
paragraph = doc.add_paragraph(f'Количество дубликатов в изначальных данных: \
{check.shape[0] - check.drop_duplicates().shape[0]} строчки')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75

# табличка
miss = df.isna().sum().reset_index()
table = doc.add_table(rows=df.shape[1] + 1, cols=2, style='Table Grid')
cell = table.cell(0, 0)
cell.paragraphs[0].add_run('Название переменной')
cell = table.cell(0, 1)
cell.paragraphs[0].add_run('Количество пропусков в данных после обработки (приведенные к широкому формату)')
for column, value, index in zip(miss['DESCRIPTION'], miss[0], range(1, miss.shape[0] + 1)):
    if column == '': column = 'TIMESTAMP'
    cell = table.cell(index, 0)
    cell.paragraphs[0].add_run(column)
    cell = table.cell(index, 1)
    cell.paragraphs[0].add_run(f'{value}')
    
# параграф про дубликаты
paragraph = doc.add_paragraph(f'Количество дубликатов в данных после обработки: \
{df.shape[0] - df.drop_duplicates().shape[0]} строчки')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75

# подглава про выбросы
head_3 = doc.add_heading('Детекция выбросов', level = 2)
head_2.alignment = WD_ALIGN_PARAGRAPH.CENTER
h_fmt = head_2.paragraph_format
h_fmt.line_spacing = 1.75

# текст с табличкой 
paragparh = doc.add_paragraph('В таблице предоставленны данные о выбросах')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75
table = doc.add_table(rows = 5, cols = 9, style = 'Table Grid')
cell = table.cell(0, 0)
cell.paragraphs[0].add_run('Название метода\Название переменной')
cell = table.cell(1, 0)
cell.paragraphs[0].add_run('z-score (абсолютный)')
cell = table.cell(2, 0)
cell.paragraphs[0].add_run('z-score (в процентах)')
cell = table.cell(3, 0)
cell.paragraphs[0].add_run('IQR (абсолютный)')
cell = table.cell(4, 0)
cell.paragraphs[0].add_run('IQR (в процентах)')
for i in range(0, 8):
    cell = table.cell(0, i + 1)
    cell.paragraphs[0].add_run(df.columns.droplevel(0)[i + 1])
    cell = table.cell(1, i + 1)
    cell.paragraphs[0].add_run(f'{len(dict_out_feat_z[df.columns[i + 1]])}')
    cell = table.cell(2, i + 1)
    cell.paragraphs[0].add_run(f'{(len(dict_out_feat_z[df.columns[i + 1]]) / df.shape[0]) * 100 :.1f} %')
    cell = table.cell(3, i + 1)
    cell.paragraphs[0].add_run(f'{len(dict_out_feat_iqr[df.columns[i + 1]][0])}')
    cell = table.cell(4, i + 1)
    cell.paragraphs[0].add_run(f'{(len(dict_out_feat_iqr[df.columns[i + 1]][0]) / df.shape[0]) * 100:.1f} %')

# новая глава: машинное обучение
head_4 = doc.add_heading('Результаты машинного обучения', level = 1)
# подглава: линейные модели
head_5 = doc.add_heading('Линейные модели', level = 2)
table = doc.add_table(metrics_linear.shape[0]+1, metrics_linear.shape[1], style='Table Grid')
for j in range(metrics_linear.shape[-1]):
    table.cell(0,j).text = metrics_linear.columns[j]
for i in range(metrics_linear.shape[0]):
    for j in range(metrics_linear.shape[-1]):
        table.cell(i+1,j).text = str(metrics_linear.values[i,j])
# подглава: деревянные модели   
head_6 = doc.add_heading('Деревянные модели', level = 2)
table = doc.add_table(metrics_tree.shape[0] + 1, metrics_tree.shape[1], style='Table Grid')
for j in range(metrics_tree.shape[-1]):
    table.cell(0,j).text = metrics_tree.columns[j]
for i in range(metrics_tree.shape[0]):
    for j in range(metrics_tree.shape[-1]):
        table.cell(i+1,j).text = str(metrics_tree.values[i,j])
        
# подглава: бустинги
head_6 = doc.add_heading('Модели бустинга', level = 2)
table = doc.add_table(metrics_boost.shape[0] + 1, metrics_boost.shape[1], style='Table Grid')
for j in range(metrics_boost.shape[-1]):
    table.cell(0,j).text = metrics_boost.columns[j]
for i in range(metrics_boost.shape[0]):
    for j in range(metrics_boost.shape[-1]):
        table.cell(i+1,j).text = str(metrics_boost.values[i,j])
# сохранение 
doc.save('Первичный осмотр.docx')

In [63]:
doc.paragraphs